In [2]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [3]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [4]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits."""



client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 13:01:08.349924


## Step 1 - Generate Mathematical Formulation

In [5]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [6]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [7]:
Markdown(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* C1 = 49000 (coefficient for X1 in the revenue function)
* C2 = 30 (coefficient for X2 in the revenue function)
* P = 150 (cost of buying and processing each hectolitre of syrup)
* H = 40 (number of units of 1.5 l COLI product produced from each hectolitre of syrup)
* J = 20 (number of units of 1.5 l PEPSA product produced from each hectolitre of syrup)
* C = 7100 (net capacity of the S1 production line in 1.5 l product units per hour)
* S = 8 (number of hours in a work shift)
* T = 5 (number of days worked per week)

Decision variables:

* X1 (number of units of 1.5 l COLI product to produce)
* X2 (number of units of 1.5 l PEPSA product to produce)

Objective function:

Maximize profit, given by the revenue function minus the cost of syrup:

Profit = C1 \* X1 - X1^2 + C2 \* X2 - 2 \* X2^2 - P \* (X1/H + X2/J)

Constraints:

1. Capacity constraint: The total number of products produced cannot exceed the capacity of the S1 production line:

X1 + X2 <= C \* S \* T

2. Minimum PEPSA production constraint: The firm must produce at least half the amount of PEPSA as COLI:

X2 >= 0.5 \* X1

3. Non-negativity constraint: The number of products produced must be non-negative:

X1 >= 0, X2 >= 0

This optimization model can be solved using linear programming techniques to find the optimal production plan for the firm.

In [8]:
print(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* C1 = 49000 (coefficient for X1 in the revenue function)
* C2 = 30 (coefficient for X2 in the revenue function)
* P = 150 (cost of buying and processing each hectolitre of syrup)
* H = 40 (number of units of 1.5 l COLI product produced from each hectolitre of syrup)
* J = 20 (number of units of 1.5 l PEPSA product produced from each hectolitre of syrup)
* C = 7100 (net capacity of the S1 production line in 1.5 l product units per hour)
* S = 8 (number of hours in a work shift)
* T = 5 (number of days worked per week)

Decision variables:

* X1 (number of units of 1.5 l COLI product to produce)
* X2 (number of units of 1.5 l PEPSA product to produce)

Objective function:

Maximize profit, given by the revenue function minus the cost of syrup:

Profit = C1 \* X1 - X1^2 + C2 \* X2 - 2 \* X2^2 - P \* (X1/H + X2/J)

Constraints:

1. Capacity constraint: The total number of products produced cannot exceed the capacity 

## Step 2 - Generate the Pyomo Code

In [9]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [10]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [11]:
Markdown(response_2_text)

Here is a Python Pyomo code to solve the optimization problem:
```python
from pyomo.environ import *

# Parameters
C1 = 49000
C2 = 30
P = 150
H = 40
J = 20
C = 7100
S = 8
T = 5

# Model
model = ConcreteModel()

# Decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)

# Objective function
model.obj = Objective(expr=C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - P * (model.X1/H + model.X2/J), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=model.X1 + model.X2 <= C * S * T)
model.min_pepsa_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal production plan:")
print(f"COLI: {model.X1.value}")
print(f"PEPSA: {model.X2.value}")
print(f"Profit: {model.obj()}")
```
Note that the code uses the GLPK solver to solve the linear programming problem. The optimal production plan and profit are printed out after the model is solved.

Also, the model assumes that the revenue function is quadratic and the cost of syrup is linear. If the actual revenue and cost functions are different, the objective function and constraints may need to be adjusted accordingly.

In [12]:
print(response_2_text)

Here is a Python Pyomo code to solve the optimization problem:
```python
from pyomo.environ import *

# Parameters
C1 = 49000
C2 = 30
P = 150
H = 40
J = 20
C = 7100
S = 8
T = 5

# Model
model = ConcreteModel()

# Decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)

# Objective function
model.obj = Objective(expr=C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - P * (model.X1/H + model.X2/J), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=model.X1 + model.X2 <= C * S * T)
model.min_pepsa_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal production plan:")
print(f"COLI: {model.X1.value}")
print(f"PEPSA: {model.X2.value}")
print(f"Profit: {model.obj()}")
```
Note that the code uses the GLPK solver to solve the linear programming problem. The optimal production plan and profit are 

### Code Executability

In [13]:
from pyomo.environ import *

# Parameters
C1 = 49000
C2 = 30
P = 150
H = 40
J = 20
C = 7100
S = 8
T = 5

# Model
model = ConcreteModel()

# Decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)

# Objective function
model.obj = Objective(expr=C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - P * (model.X1/H + model.X2/J), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=model.X1 + model.X2 <= C * S * T)
model.min_pepsa_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Optimal production plan:")
print(f"COLI: {model.X1.value}")
print(f"PEPSA: {model.X2.value}")
print(f"Profit: {model.obj()}")

Optimal production plan:
COLI: 16335.833333340022
PEPSA: 8167.9166666600495
Profit: 400289176.0419919


### Solution Correctness